In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('Listing_cleaned.csv')
df.head()

,Unnamed: 0,Address,Price,Beds,Baths,City,Area,Street,Description
0,0,2323 Confederation Pkwy 1102,430000,2,2,Mississauga,Cooksville,Confederation Pkwy,Beautiful Bright And Spacious Condo Unit 2 Bed...
1,1,816 Lansdowne Ave 411,598000,2,1,Toronto,Dovercourt-Wallace Emerson-Junction,Lansdowne Ave,Large Renovated One Bedroom+Den 646 Sf! Great ...
2,6,109 Lake St,750000,4,2,St. Catharines,NaN,NaN,Investors!! Maintenance-Free Solid 4 Plex-Full...
3,7,321 Queenston Rd,1899000,4,4,Niagara-on-the-Lake,NaN,NaN,"Welcome To 321 Queenston Rd, Overlooking Viney..."
4,8,650 Upper Sherman Ave,899000,4,2,Hamilton,Burkholme,Upper Sherman Ave,Absolutely Fantastic Opportunity !!! Detach Ho...


In [3]:
df['City'].value_counts().head(60)

Toronto                      178
Mississauga                  103
Vaughan                       85
Brampton                      79
Richmond Hill                 72
Markham                       71
Hamilton                      66
Oakville                      55
Burlington                    48
Barrie                        44
Oshawa                        40
Milton                        40
Kitchener                     37
Caledon                       34
Whitby                        31
Aurora                        30
Georgina                      29
Clarington                    29
Newmarket                     28
Pickering                     26
Kawartha Lakes                25
Niagara Falls                 24
New Tecumseth                 24
Innisfil                      24
Waterloo                      23
Whitchurch-Stouffville        22
Ajax                          22
East Gwillimbury              22
Guelph                        21
St. Catharines                21
Halton Hil

In [ ]:
df['City'] = df['City'].apply(lambda x: x if x in ('Toronto','Mississauga','Vaughan','Brampton','Richmond Hill','Markham','Hamilton','Oakville','Burlington','Barrie','Oshawa','Milton','Kitchener','Caledon','Whitby','Aurora','Georgina','Clarington','Newmarket','Pickering','Kawartha Lakes','Niagara Falls','New Tecumseth','Innisfil','Waterloo','Whitchurch-Stouffville','Ajax','East Gwillimbury','Guelph','St. Catharines','Halton Hills','Brantford','Cambridge','Blue Mountains','London','Oro-Medonte','Wasaga Beach','Bradford West Gwillimbury','Orangeville','King','Grimsby','Haldimand','Orillia','Lincoln','Essa','Ottawa','Welland','Thorold','Port Hope','Quinte West') else 'Other')

In [7]:
df1 = df['City'].value_counts() == 1

df1.value_counts()

True     219
False    143
Name: City, dtype: int64

In [ ]:
df[['half','finish']] = df[['half','finish']].apply(pd.to_timedelta).apply(lambda x: x.dt.total_seconds())

In [ ]:
X = df[['half','finish', 'division', 'country']].copy()
X['division'] = LabelEncoder().fit_transform(X['division'])
X['country'] = LabelEncoder().fit_transform(X['country'])
X = X.dropna()
X.head()

In [ ]:
# Scale the dataset using MinMaxScaler()
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled

In [ ]:
sse = {}
K = range(1, 10)
for k in K:
    kmeanmodel = KMeans(n_clusters=k).fit(X_scaled)
    sse[k]= kmeanmodel.inertia_
    
# Plot
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel('k')
plt.ylabel('SSE')
plt.title('Elbow Method')
plt.show()

In [ ]:
# Create a KMeans model with 3 clusters
model = KMeans(n_clusters=3, random_state=42).fit(X_scaled)

In [ ]:
# Calculate predicted values.
y_pred = model.predict(X_scaled)

In [ ]:
# Add prediected values onto the original dataframe
df_y = pd.DataFrame(y_pred, columns=['Cluster'])
combined = df.join(df_y, how='inner')
combined.head()

In [ ]:
# Plot boxplots of the official time distributions by cluster
combined.boxplot(['finish'], by=['Cluster'])

In [ ]:
# Plot median finish times by division for each cluster
combined[combined['Cluster'] == 0].groupby('division').median()['finish'].plot()
combined[combined['Cluster'] == 1].groupby('division').median()['finish'].plot()
combined[combined['Cluster'] == 2].groupby('division').median()['finish'].plot()